In [1]:
import torch
import pandas as pd

In [2]:
data1 = pd.read_csv("./datasets/fake_review_data/fake_reviews_dataset.csv")
data2 = pd.read_csv("./datasets/Amazon_Reviews_full-product/amazon_reviews.csv")

In [3]:
data1.head()

,category,rating,label,text
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


In [4]:
data2.head()

,DOC_ID,LABEL,RATING,VERIFIED_PURCHASE,PRODUCT_CATEGORY,PRODUCT_ID,PRODUCT_TITLE,REVIEW_TITLE,REVIEW_TEXT
0,1,__label1__,4,N,PC,B00008NG7N,"Targus PAUK10U Ultra Mini USB Keypad, Black",useful,"When least you think so, this product will sav..."
1,2,__label1__,4,Y,Wireless,B00LH0Y3NM,Note 3 Battery : Stalion Strength Replacement ...,New era for batteries,Lithium batteries are something new introduced...
2,3,__label1__,3,N,Baby,B000I5UZ1Q,"Fisher-Price Papasan Cradle Swing, Starlight",doesn't swing very well.,I purchased this swing for my baby. She is 6 m...
3,4,__label1__,4,N,Office Products,B003822IRA,Casio MS-80B Standard Function Desktop Calculator,Great computing!,I was looking for an inexpensive desk calcolat...
4,5,__label1__,4,N,Beauty,B00PWSAXAM,Shine Whitening - Zero Peroxide Teeth Whitenin...,Only use twice a week,I only use it twice a week and the results are...


In [5]:
data1["label"] = data1['label'].replace({"CG":1,"OR":0})

In [6]:
data2["label"] = data2['LABEL'].replace({"__label1__":1,"__label2__":0})
data2 = data2.drop(columns=["LABEL"])

In [7]:
def cleanCategory(x):
    return " ".join(x.split("_")[:-1])

data1['category'] = data1['category'].apply(cleanCategory)

In [8]:
data2["VERIFIED_PURCHASE"] = data2['VERIFIED_PURCHASE'].replace({"N":0,"Y":1})

data2["category"] = data2["PRODUCT_CATEGORY"] + " [SEP] " + data2["PRODUCT_TITLE"]
data2 = data2.drop(columns=["PRODUCT_TITLE","PRODUCT_CATEGORY"])

data2["text"] = data2["REVIEW_TITLE"] + " [SEP] " + data2["REVIEW_TEXT"]
data2 = data2.drop(columns=["REVIEW_TITLE","REVIEW_TEXT"])

data2['rating'] = data2['RATING']
data2 = data2.drop(columns=['RATING'])

In [9]:
data1.head()

,category,rating,label,text
0,Home and Kitchen,5.0,1,"Love this! Well made, sturdy, and very comfor..."
1,Home and Kitchen,5.0,1,"love it, a great upgrade from the original. I..."
2,Home and Kitchen,5.0,1,This pillow saved my back. I love the look and...
3,Home and Kitchen,1.0,1,"Missing information on how to use it, but it i..."
4,Home and Kitchen,5.0,1,Very nice set. Good quality. We have had the s...


In [10]:
data2.head()

,DOC_ID,VERIFIED_PURCHASE,PRODUCT_ID,label,category,text,rating
0,1,0,B00008NG7N,1,"PC [SEP] Targus PAUK10U Ultra Mini USB Keypad,...","useful [SEP] When least you think so, this pro...",4
1,2,1,B00LH0Y3NM,1,Wireless [SEP] Note 3 Battery : Stalion Streng...,New era for batteries [SEP] Lithium batteries ...,4
2,3,0,B000I5UZ1Q,1,"Baby [SEP] Fisher-Price Papasan Cradle Swing, ...",doesn't swing very well. [SEP] I purchased thi...,3
3,4,0,B003822IRA,1,Office Products [SEP] Casio MS-80B Standard Fu...,Great computing! [SEP] I was looking for an in...,4
4,5,0,B00PWSAXAM,1,Beauty [SEP] Shine Whitening - Zero Peroxide T...,Only use twice a week [SEP] I only use it twic...,4


In [11]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [12]:
DEVICE = torch.device("cuda")

In [13]:
from torch import Tensor
from torch.nn import Module,Dropout,Linear,Embedding,LayerNorm,BatchNorm1d
from torch.utils.data import DataLoader, Dataset

In [14]:
torch.autograd.set_detect_anomaly(True)

In [15]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

class CategoryReviewDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.stopwords = stopwords.words('english')
        keep = ['not','no','because','but','against','nor','very']
        for elem in keep:
            self.stopwords.remove(elem)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        txt = "[CLS] " + " " + str(row["rating"]) + " " + row["text"]
        review = [word for word in txt.split() if word.lower() not in self.stopwords]
        label = row["label"]
        category = row["category"]
        # rating = row["rating"]
        
        reviewEncoding = {}
        reviewEncoding['input_ids'] = review
        if len(reviewEncoding['input_ids'])<512:
            reviewEncoding['input_ids'].append("[EOS]")
            reviewEncoding['input_ids'] += ["[PAD]"]*(512-len(reviewEncoding['input_ids']))
        else:
            reviewEncoding['input_ids'] = reviewEncoding['input_ids'][:511]
            reviewEncoding['input_ids'].append("[EOS]")
        reviewEncoding['category'] = category.split()
        if len(reviewEncoding['category'])<82:
            reviewEncoding['category'] += ["[PAD]"]*(82-len(reviewEncoding['category']))
        else:
            reviewEncoding['category'] = reviewEncoding['category'][:82]
        # reviewEncoding['rating'] = torch.tensor(rating, dtype=torch.float32)
        reviewEncoding["label"] = torch.tensor(label)
        return reviewEncoding

[nltk_data] Downloading package stopwords to
[nltk_data]     /raid/home/akshat21515/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
def collate_fn(batch):
    text, categories, labels = [], [], []
    for sample in batch:
        text.append(sample['input_ids'])
        categories.append(sample['category'])
        # ratings.append(sample['rating'])
        labels.append(sample["label"])

    return text, categories, torch.stack(labels)

In [17]:
class PositionalEncodings(Module):
    def __init__(self, d_model:int, max_len:int=512):
        super().__init__()
        self.d_model = d_model
        self.max_len = max_len

        pe = torch.zeros((max_len, d_model))
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(torch.log(torch.tensor(10000.0)) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x:Tensor):
        return (self.pe[:x.size(1), :]).unsqueeze(0)

In [18]:
from typing import Any,Tuple
import torch.nn.functional as F

class FakeReviewIdentifier(Module):
    def __init__(self,
                 model,
                 embeddingLayer,
                 embSize:int,
                 d_model:int,
                 vocabSize:int,
                 num_classes:int,
                 dropoutRate:float,
                 actOut:Any):
        
        super().__init__()
        self.embedding = embeddingLayer
        self.transform = Linear(embSize,d_model)
        self.positionalEncoding = PositionalEncodings(d_model)
        self.vocabSize = vocabSize

        self.model = model
        self.dropout = Dropout(dropoutRate)
        self.layerNorm = LayerNorm(d_model)

        self.fc1 = Linear(d_model+1,d_model//2)
        self.fc2 = Linear(d_model//2,d_model//4)
        self.fc3 = Linear(d_model//4,num_classes)

        self.act1 = F.tanh
        self.act2 = actOut

    def forward(self, text, category, attnMask):
        x = self.embedding(text)
        x = x.to(DEVICE)
        x = self.transform(x)
        
        xCategory = self.embedding(category)
        xCategory = xCategory.to(DEVICE)
        xCategory = self.transform(xCategory)
        
        x += self.positionalEncoding(x)
        xCategory += self.positionalEncoding(xCategory)

        xOut = self.model(x,xCategory,attnMask)
        xOut = xOut[:,0,:]
        # print(xOut.shape)
        # xOut = torch.stack([xOut,torch.tensor(rating/5.0).unsqueeze_(0)])
        xOut = xOut.to(torch.float32)
        
        out = self.fc1(xOut)
        out = self.fc2(out)
        out = self.act2(out)
        out = self.fc3(out)
        out = torch.softmax(out,dim=-1,dtype=torch.float32)

        return out

In [19]:
train_data = data2.sample(frac=0.8)
val_data = data2.drop(train_data.index)
test_data = val_data.sample(frac=0.5)

In [20]:
from transformers import AutoTokenizer,AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

train_loader = DataLoader(CategoryReviewDataset(train_data), batch_size=64, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(CategoryReviewDataset(val_data), batch_size=64, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(CategoryReviewDataset(test_data), batch_size=64, shuffle=True, collate_fn=collate_fn)

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/raid/home/akshat21515/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
class CategoryAttention(Module):
    def __init__(self,
                 d_model:int=512,
                 n_heads:int=8):
        super().__init__()
        assert d_model % n_heads == 0, "d_model must be divisible by n_head"
        self.d_model = d_model
        self.n_heads = n_heads
        self.head_dim = d_model//n_heads

        self.Wk = Linear(d_model,d_model)
        self.Wq = Linear(d_model,d_model)
        self.Wv = Linear(d_model,d_model)
        self.Wo = Linear(d_model,d_model)

    def forward(self,query,key,value,mask=None):
        batch_size = query.size(0)

        Q = self.Wq(query)
        K = self.Wk(key)
        V = self.Wv(value)

        Q = Q.view(batch_size,-1,self.n_heads,self.head_dim).permute(0,2,1,3)
        K = K.view(batch_size,-1,self.n_heads,self.head_dim).permute(0,2,1,3)
        V = V.view(batch_size,-1,self.n_heads,self.head_dim).permute(0,2,1,3)

        energy = torch.matmul(Q,K.permute(0,1,3,2)) / torch.sqrt(torch.tensor(self.head_dim,dtype=torch.float32))
        if mask is not None:
            energy.masked_fill_(mask==0,float("-1e20"))

        attnWeights = torch.softmax(energy,dim=1)
        out = torch.matmul(attnWeights,V)
        out = out.permute(0,2,1,3).contiguous().view(batch_size,-1,self.d_model)

        out = self.Wo(out)
        return out

In [22]:
class TransformerEncoderLayer(Module):
    def __init__(self,
                 d_model:int=512,
                 n_heads:int=8,
                 dim_feedforward:int=2048,
                 dropout:float=0.1,
                 activation:Any = F.relu):
        
        super().__init__()
        self.selfAttn = CategoryAttention(d_model,n_heads)
        
        self.fc1 = Linear(d_model,dim_feedforward)
        self.dropout = Dropout(dropout)
        
        self.fc2 = Linear(dim_feedforward,d_model)
        self.norm1 = LayerNorm(d_model)
        self.norm2 = LayerNorm(d_model)

        self.dropout1 = Dropout(dropout)
        self.dropout2 = Dropout(dropout)
        self.dropout3 = Dropout(dropout)
        
        self.activation = activation

    def forward(self,
                sent1:Tensor,
                attnMask:Tensor=None,
                crossAttn:Tensor=None):
        
        attnSet = self.selfAttn(sent1,sent1,sent1,attnMask)
        resConn = self.dropout1(attnSet)
        if crossAttn is not None:
            resConn += self.dropout2(crossAttn)
        normOut = self.norm1(resConn)

        FFNOutp = self.activation(self.fc1(normOut))
        FFNProd = self.fc2(self.dropout(FFNOutp))

        out = resConn + self.dropout3(FFNProd)
        out = self.norm2(out)
        return out

In [23]:
from torch.nn import ModuleList

class TransformerEncoder(Module):
    def __init__(self,
                 encoder_layer:Module,
                 num_layers:int=8,
                 d_model:int=512,
                 n_heads:int=8,
                 dim_feedforward:int=2048,
                 dropout:float=0.1,
                 activation:Any=F.relu):
        
        super().__init__()
        self.catAttn = CategoryAttention(d_model,n_heads)
        self.layers = ModuleList([encoder_layer(d_model,n_heads,dim_feedforward,dropout,activation) for _ in range(num_layers)])

    def forward(self,
                sent1:Tensor,
                sent2:Tensor,
                mask:Tuple[Tensor]=None):
        out = sent1
        attnCat = self.catAttn(sent1,sent2,sent2,mask[0])
        for layer in self.layers:
            out = layer(out,mask[1],attnCat)
        return out

In [24]:
from gensim.models.fasttext import load_facebook_vectors

def FasttextEmbedding(sentences,model=load_facebook_vectors("cc.en.300.bin")):
    embeds = []
    for sentence in sentences:
        wordEmbeds = []
        for i in range(len(sentence)):
            wordEmbeds.append(torch.tensor(model[sentence[i]]))
        if wordEmbeds:
            sentenceEmbedding = torch.stack(wordEmbeds)
            embeds.append(sentenceEmbedding)
    return torch.stack(embeds)

In [25]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
import torch.nn as nn

embSize = 300
d_model = 128
n_heads = 16
num_layers = 10
num_classes = 2
vocab_size = len(tokenizer)
dropout_rate = 0.25
dim_feedforward = 512
activation = F.leaky_relu_

baseModel = TransformerEncoder(
    TransformerEncoderLayer,
    num_layers,
    d_model,
    n_heads,
    dim_feedforward,
    dropout_rate,
    activation
)

model = FakeReviewIdentifier(baseModel,FasttextEmbedding,embSize,d_model,vocab_size,num_classes,dropout_rate,F.leaky_relu_)

param_list = list(model.parameters()) + list(baseModel.parameters())
for param in param_list:
    if param.dim()>1:
        nn.init.xavier_uniform_(param)

optimizer = Adam(param_list, lr=1e-4)
loss_fn = CrossEntropyLoss()

/raid/home/akshat21515/.local/lib/python3.8/site-packages/torch/_compile.py:24: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  return torch._dynamo.disable(fn, recursive)(*args, **kwargs)


In [26]:
def generateMask(t1,t2):
    attnMask = []
    for i in range(len(t1)):
        s1 = t1[i]
        curMask = []
        for word in s1:
            if word=="[PAD]":
                curMask.append(0)
            else:
                curMask.append(1)
        s1Mask = torch.tensor(curMask).unsqueeze_(1)

        s2 = t2[i]
        curMask = []
        for word in s2:
            if word=="[PAD]":
                curMask.append(0)
            else:
                curMask.append(1)
        s2Mask = torch.tensor(curMask).unsqueeze_(0)
        attnMask.append(torch.matmul(s1Mask,s2Mask))
    return torch.stack(attnMask).unsqueeze_(1)

In [27]:
def epoch(loader, model, loss_fn, optimizer, is_train:bool=False):
    model.train(is_train)
    total_loss = 0
    total_acc = 0
    for text, category, labels in loader:
        # rating = rating.view(-1,1).to(DEVICE)
        labels = labels.to(DEVICE)

        with torch.set_grad_enabled(is_train):
            out = model(text, category, attnMask=(generateMask(text,category).to(DEVICE),generateMask(text,text).to(DEVICE)))
            loss = loss_fn(out, labels)
            _,preds = torch.max(out,dim=1)
            acc = (preds==labels).sum().item()/labels.size(0)
            if is_train:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            total_loss += loss.item()
            total_acc += acc
    return total_loss / len(loader), total_acc / len(loader)

In [28]:
torch.cuda.empty_cache()

In [29]:
num_epochs = 100
train_losses, val_losses, train_accs, val_accs = [], [], [], []
patience = 10
min_val_loss = 1e9
bad_epochs = 0

for currEpoch in range(num_epochs):
    model.to(DEVICE)
    
    train_loss,train_acc = epoch(train_loader, model, loss_fn, optimizer, is_train=True)
    val_loss,val_acc = epoch(val_loader, model, loss_fn, optimizer, is_train=False)

    print(f"Epoch {currEpoch+1}")
    print(f"Train Loss: {train_loss:.4f} Train Accuracy: {train_acc:.4f} | Val Loss: {val_loss:.4f} Val Accuracy: {val_acc:.4f}")

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)

    if val_loss < min_val_loss:
        min_val_loss = val_loss
        bad_epochs = 0
    else:
        bad_epochs += 1
    if bad_epochs == patience:
        break

Epoch 1
Train Loss: 0.6854 Train Accuracy: 0.5513 | Val Loss: 0.6686 Val Accuracy: 0.5682
Epoch 2
Train Loss: 0.6675 Train Accuracy: 0.5827 | Val Loss: 0.6687 Val Accuracy: 0.5792
Epoch 3
Train Loss: 0.6648 Train Accuracy: 0.5874 | Val Loss: 0.6666 Val Accuracy: 0.5870
Epoch 4
Train Loss: 0.6633 Train Accuracy: 0.5896 | Val Loss: 0.6631 Val Accuracy: 0.5867
Epoch 5
Train Loss: 0.6596 Train Accuracy: 0.5955 | Val Loss: 0.6593 Val Accuracy: 0.5937
Epoch 6
Train Loss: 0.6540 Train Accuracy: 0.6068 | Val Loss: 0.6507 Val Accuracy: 0.6147
Epoch 7
Train Loss: 0.6480 Train Accuracy: 0.6180 | Val Loss: 0.6485 Val Accuracy: 0.6266
Epoch 8
Train Loss: 0.6409 Train Accuracy: 0.6306 | Val Loss: 0.6450 Val Accuracy: 0.6320
Epoch 9
Train Loss: 0.6332 Train Accuracy: 0.6445 | Val Loss: 0.6363 Val Accuracy: 0.6400
Epoch 10
Train Loss: 0.6241 Train Accuracy: 0.6561 | Val Loss: 0.6317 Val Accuracy: 0.6439
Epoch 11
Train Loss: 0.6260 Train Accuracy: 0.6568 | Val Loss: 0.6681 Val Accuracy: 0.6049
Epoch 12